In [1]:
!pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 4.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.10-cp312-cp312-macosx_11_0_arm64.whl size=133407 sha256=1c2cf1cad22b30676c4e8a41efdba70c3c8c44bbe821cc3af6312b8ec5b9e633
  Stored in directory: /Users/paramasivandorai/Library/Caches/pip/wheels/ac/bb/ce/afa589c50b6004d3a06fc691e71bd09c9bd5f01e5921e5329b
Successfully built psycopg2


In [9]:
import psycopg2
import pandas as pd

# Load the CSV data into a pandas DataFrame
df = pd.read_csv('/Users/paramasivandorai/desktop/Personal Projects/RetailDataset/Category.csv')

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="P0stGr3sM6ntr6",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Insert data into the Category table
for _, row in df.iterrows():
    cursor.execute(
        "INSERT INTO Category (category_id, category_name, comment) VALUES (%s, %s, %s)",
        (row['category_id'], row['category_name'], row['comment'])
    )

# Commit the transaction and close the connection
conn.commit()
cursor.close()
conn.close()

In [23]:
import random
# Load the CSV data into a DataFrame
df_product = pd.read_csv('/Users/paramasivandorai/desktop/Personal Projects/RetailDataset/Product.csv')

# Categories and their associated ranges
category_ranges = {
    "range_10_100": [1000, 1003, 1004, 1006, 1007, 1020],
    "range_1000_20000": [1001, 1010, 1011, 1016, 1012, 1017, 1018],
    "range_100_5000": [1002, 1008, 1014, 1007, 1009, 1020, 1019]
}

# Set average_cost based on category ranges
def set_average_cost(row):
    if row['category_id'] in category_ranges["range_10_100"]:
        return round(random.uniform(10, 100), 2)
    elif row['category_id'] in category_ranges["range_1000_20000"]:
        return round(random.uniform(1000, 20000), 2)
    elif row['category_id'] in category_ranges["range_100_5000"]:
        return round(random.uniform(100, 5000), 2)
    else:
        return round(random.uniform(100, 2000), 2)

# Calculate the unit_retail with a 70% markup
def calculate_unit_retail(average_cost):
    markup_price = average_cost * 1.70  # Apply a 70% markup
    return round(markup_price * 4) / 4  # Round to nearest 0.05 (common tender value)

# Apply the functions to set average_cost and calculate unit_retail
df_product['average_cost'] = df_product.apply(set_average_cost, axis=1)
df_product['unit_retail'] = df_product['average_cost'].apply(calculate_unit_retail)

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="P0stGr3sM6ntr6",
    host="localhost",
    port="5432"
)

cursor = conn.cursor()

# Insert data into the Product table
for _, row in df_product.iterrows():
    cursor.execute(
        """
        INSERT INTO Product (product_id, product_title, product_description, category_id, average_cost, unit_retail, promotional_price)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """,
        (row['product_id'], row['product_title'], row['product_description'], row['category_id'], row['average_cost'], row['unit_retail'], row['promotional_price'])
    )

# Commit the transaction and close the connection
conn.commit()
cursor.close()
conn.close()